<a href="https://colab.research.google.com/github/chethana613/LLM-Readings-and-Assignments/blob/main/LLMAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00


In [ ]:
#all emotions taken into account
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from collections import defaultdict

# Load the dataset
dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split='test')

# Load the BLOOM tokenizer and model
#model_name = "bigscience/bloom-560m"
model_name = "bigscience/bloom-1b1"
model_name = "bigscience/bloom-1b7"
model_name = "bigscience/bloomz-560m"
model_name = "bigscience/bloomz-1b1"
model_name = "bigscience/bloomz-1b7"

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = AutoModelForSequenceClassification.from_pretrained("bigscience/bloom-560m")

# Define the task labels
labels = ["negative", "neutral", "positive"]

# Define prompts in the format "label": "prompt"
prompts = {
  "positive": "This tweet expresses joy and positivity.",
    "negative": "The following tweet conveys sadness and disappointment.",
    "neutral": "The sentiment of this tweet is neutral and lacks strong emotions."
}

# Zero-shot inference
zero_shot_predictions = []
true_labels = []

for text, label_id in zip(dataset['text'], dataset['label']):
    true_label = labels[label_id]
    true_labels.append(true_label)

    prompt_votes = defaultdict(int)
    for prompt_label, prompt in prompts.items():
        inputs = tokenizer(prompt + ": " + text, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        predicted_label_idx = torch.argmax(outputs.logits, dim=1).item()
        prompt_votes[prompt_label] += predicted_label_idx  # Counting all predictions

    # Select the prompt with the majority vote
    majority_prompt = max(prompt_votes, key=prompt_votes.get)
    zero_shot_predictions.append(majority_prompt)

# Calculate zero-shot metrics
zero_shot_accuracy = accuracy_score(true_labels, zero_shot_predictions)
zero_shot_metrics = precision_recall_fscore_support(true_labels, zero_shot_predictions, average='weighted')
print(f"Zero-shot Accuracy: {zero_shot_accuracy}")
print(f"Zero-shot Precision: {zero_shot_metrics[0]}")
print(f"Zero-shot Recall: {zero_shot_metrics[1]}")
print(f"Zero-shot F1-score: {zero_shot_metrics[2]}")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for cardiffnlp/tweet_sentiment_multilingual contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cardiffnlp/tweet_sentiment_multilingual
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Zero-shot Accuracy: 0.3333333333333333
Zero-shot Precision: 0.1111111111111111
Zero-shot Recall: 0.3333333333333333
Zero-shot F1-score: 0.16666666666666666


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
pip install tqdm

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report
import torch
from collections import defaultdict
from tqdm import tqdm

# Load the dataset
dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split='test')

model_name = "bigscience/bloom-560m"
#model_name = "bigscience/bloom-1b1"
#model_name = "bigscience/bloom-1b7"
#model_name = "bigscience/bloomz-560m"
#model_name = "bigscience/bloomz-1b1"
#model_name = "bigscience/bloomz-1b7"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["negative", "neutral", "positive"]

prompts = {
    "positive": "This tweet expresses joy and positivity.",
    "negative": "The following tweet conveys sadness and disappointment.",
    "neutral": "The sentiment of this tweet is neutral and lacks strong emotions."
}

zero_shot_predictions = []
true_labels = []

# Wrap the loop with tqdm
for text, label_id in tqdm(zip(dataset['text'], dataset['label']), total=len(dataset)):
    true_label = labels[label_id]
    true_labels.append(true_label)

    prompt_votes = defaultdict(int)
    for prompt_label, prompt in prompts.items():
        inputs = tokenizer(prompt + " " + text, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        predicted_label_idx = torch.argmax(outputs.logits, dim=1).item()
        prompt_votes[prompt_label] += predicted_label_idx

    majority_prompt = max(prompt_votes, key=prompt_votes.get)
    zero_shot_predictions.append(majority_prompt)

zero_shot_accuracy = accuracy_score(true_labels, zero_shot_predictions)
zero_shot_classification_report = classification_report(true_labels, zero_shot_predictions, target_names=labels)

print(f"Zero-shot Accuracy: {zero_shot_accuracy}")
print("Zero-shot Classification Report:")
print(zero_shot_classification_report)


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report
import torch
from collections import defaultdict
from tqdm import tqdm

# Load the dataset
dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split='test')

#model_name = "bigscience/bloom-560m"
#model_name = "bigscience/bloom-1b1"
#model_name = "bigscience/bloom-1b7"
model_name = "bigscience/bloomz-560m"
#model_name = "bigscience/bloomz-1b1"
#model_name = "bigscience/bloomz-1b7"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["negative", "neutral", "positive"]

prompts = {
    "positive": "This tweet expresses joy and positivity.",
    "negative": "The following tweet conveys sadness and disappointment.",
    "neutral": "The sentiment of this tweet is neutral and lacks strong emotions."
}

zero_shot_predictions = []
true_labels = []

# Wrap the loop with tqdm
for text, label_id in tqdm(zip(dataset['text'], dataset['label']), total=len(dataset)):
    true_label = labels[label_id]
    true_labels.append(true_label)

    prompt_votes = defaultdict(int)
    for prompt_label, prompt in prompts.items():
        inputs = tokenizer(prompt + " " + text, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        predicted_label_idx = torch.argmax(outputs.logits, dim=1).item()
        prompt_votes[prompt_label] += predicted_label_idx

    majority_prompt = max(prompt_votes, key=prompt_votes.get)
    zero_shot_predictions.append(majority_prompt)

zero_shot_accuracy = accuracy_score(true_labels, zero_shot_predictions)
zero_shot_classification_report = classification_report(true_labels, zero_shot_predictions, target_names=labels)

print(f"Zero-shot Accuracy: {zero_shot_accuracy}")
print("Zero-shot Classification Report:")
print(zero_shot_classification_report)


Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloomz-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 870/870 [26:32<00:00,  1.83s/it]

Zero-shot Accuracy: 0.32988505747126434
Zero-shot Classification Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       290
     neutral       0.00      0.00      0.00       290
    positive       0.33      0.99      0.50       290

    accuracy                           0.33       870
   macro avg       0.11      0.33      0.17       870
weighted avg       0.11      0.33      0.17       870



In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report
import torch
from collections import defaultdict
from tqdm import tqdm

# Load the dataset
dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split='test')

#model_name = "bigscience/bloom-560m"
model_name = "bigscience/bloom-1b1"
#model_name = "bigscience/bloom-1b7"
#model_name = "bigscience/bloomz-560m"
#model_name = "bigscience/bloomz-1b1"
#model_name = "bigscience/bloomz-1b7"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["negative", "neutral", "positive"]

prompts = {
    "positive": "This tweet expresses joy and positivity.",
    "negative": "The following tweet conveys sadness and disappointment.",
    "neutral": "The sentiment of this tweet is neutral and lacks strong emotions."
}

zero_shot_predictions = []
true_labels = []

# Wrap the loop with tqdm
for text, label_id in tqdm(zip(dataset['text'], dataset['label']), total=len(dataset)):
    true_label = labels[label_id]
    true_labels.append(true_label)

    prompt_votes = defaultdict(int)
    for prompt_label, prompt in prompts.items():
        inputs = tokenizer(prompt + " " + text, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        predicted_label_idx = torch.argmax(outputs.logits, dim=1).item()
        prompt_votes[prompt_label] += predicted_label_idx

    majority_prompt = max(prompt_votes, key=prompt_votes.get)
    zero_shot_predictions.append(majority_prompt)

zero_shot_accuracy = accuracy_score(true_labels, zero_shot_predictions)
zero_shot_classification_report = classification_report(true_labels, zero_shot_predictions, target_names=labels)

print(f"Zero-shot Accuracy: {zero_shot_accuracy}")
print("Zero-shot Classification Report:")
print(zero_shot_classification_report)


model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-1b1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 870/870 [58:28<00:00,  4.03s/it]

Zero-shot Accuracy: 0.32068965517241377
Zero-shot Classification Report:
              precision    recall  f1-score   support

    negative       0.25      0.07      0.11       290
     neutral       0.50      0.00      0.01       290
    positive       0.33      0.89      0.48       290

    accuracy                           0.32       870
   macro avg       0.36      0.32      0.20       870
weighted avg       0.36      0.32      0.20       870



In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report
import torch
from collections import defaultdict
from tqdm import tqdm

# Load the dataset
dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split='test')

#model_name = "bigscience/bloom-560m"
#model_name = "bigscience/bloom-1b1"
#model_name = "bigscience/bloom-1b7"
#model_name = "bigscience/bloomz-560m"
model_name = "bigscience/bloomz-1b1"
#model_name = "bigscience/bloomz-1b7"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["negative", "neutral", "positive"]

prompts = {
    "positive": "This tweet expresses joy and positivity.",
    "negative": "The following tweet conveys sadness and disappointment.",
    "neutral": "The sentiment of this tweet is neutral and lacks strong emotions."
}

zero_shot_predictions = []
true_labels = []

# Wrap the loop with tqdm
for text, label_id in tqdm(zip(dataset['text'], dataset['label']), total=len(dataset)):
    true_label = labels[label_id]
    true_labels.append(true_label)

    prompt_votes = defaultdict(int)
    for prompt_label, prompt in prompts.items():
        inputs = tokenizer(prompt + " " + text, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        predicted_label_idx = torch.argmax(outputs.logits, dim=1).item()
        prompt_votes[prompt_label] += predicted_label_idx

    majority_prompt = max(prompt_votes, key=prompt_votes.get)
    zero_shot_predictions.append(majority_prompt)

zero_shot_accuracy = accuracy_score(true_labels, zero_shot_predictions)
zero_shot_classification_report = classification_report(true_labels, zero_shot_predictions, target_names=labels)

print(f"Zero-shot Accuracy: {zero_shot_accuracy}")
print("Zero-shot Classification Report:")
print(zero_shot_classification_report)


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloomz-1b1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 870/870 [57:42<00:00,  3.98s/it]

Zero-shot Accuracy: 0.36666666666666664
Zero-shot Classification Report:
              precision    recall  f1-score   support

    negative       0.18      0.02      0.04       290
     neutral       0.36      0.27      0.31       290
    positive       0.38      0.81      0.51       290

    accuracy                           0.37       870
   macro avg       0.31      0.37      0.29       870
weighted avg       0.31      0.37      0.29       870

